In [1]:
print("OK")

OK


In [2]:
import pandas as pd

In [26]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain_pinecone import PineconeVectorStore
from langchain.schema import Document
from langchain.vectorstores import FAISS

In [4]:
# Load the dataset
file_path = '../data/Large_Website_Development_Agency_Chatbot_Dataset.csv'
data = pd.read_csv(file_path)

In [8]:
# Preview the data
print(data.head())

                                          User Query  \
0  What platforms do you work with? Can you help me?   
1  I need help with what platforms do you work wi...   
2  Can I update my website content? Can you help me?   
3  Do you have examples of past work? Please prov...   
4      Do you offer do you provide ongoing support??   

                                            Response  
0  We work with various platforms, including Word...  
1  We work with various platforms, including Word...  
2  We can build your site with a content manageme...  
3  Yes, you can view our portfolio on our website...  
4  Yes, we offer ongoing support and maintenance ...  


In [9]:
# Check for missing values and drop them
data.dropna(inplace=True)

In [10]:
# Convert the columns to lists
queries = data['User Query'].tolist()
responses = data['Response'].tolist()

In [38]:
# Create combined documents for each query and response
combined_documents = [
    Document(
        page_content=f"Query: {queries[i]} Response: {responses[i]}",
        metadata={"index": i, "type": "query-response"}
    )
    for i in range(len(queries))
]

In [39]:
combined_documents

[Document(metadata={'index': 0, 'type': 'query-response'}, page_content="Query: What platforms do you work with? Can you help me? Response: We work with various platforms, including WordPress, Shopify, Magento, and custom solutions. Let's get started."),
 Document(metadata={'index': 1, 'type': 'query-response'}, page_content='Query: I need help with what platforms do you work with?. Response: We work with various platforms, including WordPress, Shopify, Magento, and custom solutions. Our team is ready to assist.'),
 Document(metadata={'index': 2, 'type': 'query-response'}, page_content="Query: Can I update my website content? Can you help me? Response: We can build your site with a content management system (CMS) that allows you to easily update content yourself. Let's get started."),
 Document(metadata={'index': 3, 'type': 'query-response'}, page_content="Query: Do you have examples of past work? Please provide more details. Response: Yes, you can view our portfolio on our website to 

In [16]:
# download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [17]:
embeddings = download_hugging_face_embeddings()

c:\Users\ANKIT SINGH\anaconda3\envs\chatbot\lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\ANKIT SINGH\anaconda3\envs\chatbot\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [19]:
import os
os.environ["PINECONE_API_KEY"] = "d4ac0dc4-c8bd-44e4-8d66-94043f8c8c59"

In [20]:
index_name = "customer-chatbot"

In [40]:
dosearch = PineconeVectorStore.from_documents(documents=combined_documents,
    embedding=embeddings,
    index_name=index_name)

In [41]:
query = "Can I update my website content?"
docs = dosearch.similarity_search(query, k=3)
print("Result",docs)

Result [Document(metadata={'index': 24.0, 'type': 'query-response'}, page_content='Query: Do you offer can i update my website content?? Response: We can build your site with a content management system (CMS) that allows you to easily update content yourself. Contact us for more info.'), Document(metadata={'index': 59.0, 'type': 'query-response'}, page_content="Query: Can I update my website content? Please provide more details. Response: We can build your site with a content management system (CMS) that allows you to easily update content yourself. We're here to help."), Document(metadata={'index': 20.0, 'type': 'query-response'}, page_content='Query: Can I update my website content? Response: We can build your site with a content management system (CMS) that allows you to easily update content yourself.')]


In [42]:
docs[0]

Document(metadata={'index': 24.0, 'type': 'query-response'}, page_content='Query: Do you offer can i update my website content?? Response: We can build your site with a content management system (CMS) that allows you to easily update content yourself. Contact us for more info.')

In [47]:
response = docs[0].page_content.split("Response:", 1)[1].strip()

In [48]:
print(response)

We can build your site with a content management system (CMS) that allows you to easily update content yourself. Contact us for more info.


In [49]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [50]:
prompt = PromptTemplate(template=prompt_template,
                        input_variables=["context", "question"])
chain_type_kwargs={"prompt": prompt,"verbose": True}

In [52]:
llm = CTransformers(model=r"E:\GTEC\customer-chatbot\model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens':512,
                            'temperature':0.8})

In [53]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=dosearch.as_retriever(search_kwargs={'k': 2}),
    chain_type="stuff",
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True)

In [57]:
while True:
    user_input = input(f"Input Prompt")
    result=qa({"query": user_input})
    response = result.page_content.split("Response:", 1)[1].strip()
    print("Response : ",response["result"])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: Query: Do you offer can i update my website content?? Response: We can build your site with a content management system (CMS) that allows you to easily update content yourself. Contact us for more info.

Query: Can I update my website content? Please provide more details. Response: We can build your site with a content management system (CMS) that allows you to easily update content yourself. We're here to help.
Question: Can I update my website content?

Only return the helpful answer below and nothing else.
Helpful answer:


> Finished chain.

> Finished chain.


AttributeError: 'dict' object has no attribute 'page_content'

In [59]:
while True:
    # Take user input
    user_input = input("Input Prompt: ")
    
    # Query the QA system with the user's input
    result = qa({"query": user_input})
    
    # Extract the response content from the result
    if result['source_documents']:
        # Access the first source document's content and split to extract the response
        response_text = result['source_documents'][0].page_content.split("Response:", 1)[1].strip()
        print("Response:", response_text)
    else:
        print("No relevant response found.")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: Query: Do you offer can i update my website content?? Response: We can build your site with a content management system (CMS) that allows you to easily update content yourself. Contact us for more info.

Query: Can I update my website content? Please provide more details. Response: We can build your site with a content management system (CMS) that allows you to easily update content yourself. We're here to help.
Question: Can I update my website content?"

Only return the helpful answer below and nothing else.
Helpful answer:

